# Lab0: Settings
<br>

<img src='https://github.com/retkowsky/images/blob/master/AzureMLservicebanniere.png?raw=true'>


Documentation : https://docs.microsoft.com/en-us/azure/machine-learning/

In [1]:
import sys
print("Python :", sys.version)

Python : 3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31) 
[GCC 7.3.0]


In [2]:
import datetime
now = datetime.datetime.now()
print('Date :', now)

Date : 2020-11-17 08:07:43.922962


In [3]:
import platform,socket,re,uuid,json,psutil,logging

def getSystemInfo():
    try:
        info={}
        info['Platforme']=platform.system()
        info['Platforme-release']=platform.release()
        info['Platforme-version']=platform.version()
        info['Architecture']=platform.machine()
        info['Hostname']=socket.gethostname()
        info['IP-address']=socket.gethostbyname(socket.gethostname())
        info['MAC-address']=':'.join(re.findall('..', '%012x' % uuid.getnode()))
        info['Processeur']=platform.processor()
        info['RAM']=str(round(psutil.virtual_memory().total / (1024.0 **3)))+" Go"
        return json.dumps(info)
    except Exception as e:
        logging.exception(e)

json.loads(getSystemInfo())

{'Platforme': 'Linux',
 'Platforme-release': '4.15.0-1098-azure',
 'Platforme-version': '#109~16.04.1-Ubuntu SMP Wed Sep 30 18:53:14 UTC 2020',
 'Architecture': 'x86_64',
 'Hostname': 'instancenotebooksr',
 'IP-address': '127.0.0.1',
 'MAC-address': '00:0d:3a:25:28:9c',
 'Processeur': 'x86_64',
 'RAM': '27 Go'}

In [4]:
from azureml.core import Workspace, Environment

In [5]:
import azureml.core
print("Azure ML =",azureml.core.VERSION)

Azure ML = 1.18.0


In [6]:
import scipy, numpy, matplotlib, pandas, sklearn
print("Versions :")
print('- Python = {}'.format(sys.version))
print('- scipy = {}'.format(scipy.__version__))
print('- numpy = {}'.format(numpy.__version__))
print('- matplotlib = {}'.format(matplotlib.__version__))
print('- pandas = {}'.format(pandas.__version__))
print('- sklearn = {}'.format(sklearn.__version__))
print("- Azure ML =",azureml.core.VERSION)

Versions :
- Python = 3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31) 
[GCC 7.3.0]
- scipy = 1.5.2
- numpy = 1.18.5
- matplotlib = 3.2.1
- pandas = 0.25.3
- sklearn = 0.22.2.post1
- Azure ML = 1.18.0


## Workspace access

In [7]:
import os
subscription_id = os.environ.get("SUBSCRIPTION_ID", "TOBEREPLACED") #ID soubscription Azure
resource_group = os.environ.get("RESOURCE_GROUP", "TOBEREPLACED") #Resource group
workspace_name = os.environ.get("WORKSPACE_NAME", "TOBEREPLACED") #Nom workspace

In [10]:
from azureml.core import Workspace

try:
   ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
   ws.write_config()
   print("OK. Workspace found")
except:
   print("Error")

OK. Workspace found


## Workspace information

In [12]:
ws = Workspace.from_config()
print('Workspace : ' + ws.name, 
      'Region : ' + ws.location, 
      'Ressouce Group : ' + ws.resource_group, sep='\n')

Workspace : AMLworkshop
Region : westeurope
Ressouce Group : AMLworkshop-rg


<img src="https://github.com/retkowsky/images/blob/master/Powered-by-MS-Azure-logo-v2.png?raw=true" height="300" width="300">